In [13]:
import csv
import random
from faker import Faker
from datetime import timedelta, datetime
from collections import defaultdict, Counter

### CSV EMPLEADOS

In [14]:
fake = Faker('es_ES')

In [15]:
num_empleados = 150
roles = ['Dependiente', 'Jefe de sección', 'Jefe de tienda', 'Director de tienda', 'Gerente de tienda', 'Subdirector de tienda', 'Encargado de tienda']
roles_criticos = ['Jefe de tienda', 'Director de tienda', 'Gerente de tienda']
opciones_tf = ['Sí', 'No']

empleados_data = []

with open('../data/empleados.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(['id_empleado', 'nombre_completo', 'edad_empleado', 'antiguedad_anos', 'personas_a_cargo', 'rol_empleado', 'es_rol_critico', 'tiene_ausencias_frecuentes', 'minusvalia', 'activo', 'id_departamento'])

    for i in range(num_empleados):
        nombre = fake.first_name()
        apellidos = fake.last_name() + ' ' + fake.last_name()
        edad = random.randint(18, 65)
        antiguedad_maxima = max(0, edad - 18)
        antiguedad = random.randint(0, antiguedad_maxima)
        personas_a_cargo = random.randint(0, 3)
        rol = random.choice(roles)
        es_rol_critico = 'Sí' if rol in roles_criticos else 'No'
        tiene_ausencias_frecuentes = 'Sí' if random.random() < 0.1 else 'No'
        minusvalia = random.choice(opciones_tf)
        activo = random.choice(opciones_tf)
        id_departamento = random.randint(1, 7)

        empleado = {
            'id_empleado': i + 1,
            'nombre_completo': f'{nombre} {apellidos}',
            'edad_empleado': edad,
            'rol_empleado': rol,
            'es_rol_critico': es_rol_critico == 'Sí',
            'antiguedad_anos': antiguedad,
            'personas_a_cargo': personas_a_cargo > 0,
            'tiene_ausencias_frecuentes': tiene_ausencias_frecuentes == 'Sí',
            'minusvalia': minusvalia,
            'activo': activo,
            'id_departamento': id_departamento
        }
        empleados_data.append(empleado)
        writer.writerow([i + 1, f'{nombre} {apellidos}', edad, antiguedad, personas_a_cargo, rol, es_rol_critico, tiene_ausencias_frecuentes, minusvalia, activo, id_departamento])

print("El archivo 'empleados.csv' ha sido creado exitosamente.")

El archivo 'empleados.csv' ha sido creado exitosamente.


### CSV DEPARTAMENTOS

In [16]:
nombres_departamentos = [
    "Recursos Humanos",
    "Ventas",
    "Marketing",
    "Desarrollo de Producto",
    "Atención al Cliente",
    "Finanzas",
    "Logística"
]
departamentos_data = {}
empleados_por_departamento = defaultdict(int)

for empleado in empleados_data:
    empleados_por_departamento[empleado['id_departamento']] += 1

with open('../data/departamentos.csv', 'w', newline='', encoding='utf-8') as archivo_departamentos:
    writer_departamentos = csv.writer(archivo_departamentos)
    writer_departamentos.writerow(['id_departamento', 'nombre_departamento', 'porcentaje_maximo_inactivos', 'numero_total_empleados'])

    for i, nombre in enumerate(nombres_departamentos, start=1):
        porcentaje_maximo_inactivos = round(random.uniform(0.1, 0.3), 2) * 100
        num_empleados_departamento = empleados_por_departamento[i]
        departamento = {
            'id_departamento': i,
            'nombre_departamento': nombre,
            'porcentaje_maximo_inactivos': porcentaje_maximo_inactivos,
            'numero_total_empleados': num_empleados_departamento
        }
        departamentos_data[i] = departamento
        writer_departamentos.writerow([i, nombre, int(porcentaje_maximo_inactivos), num_empleados_departamento])

print("El archivo 'departamentos.csv' ha sido creado exitosamente.")

El archivo 'departamentos.csv' ha sido creado exitosamente.


### CSV VACACIONES

In [17]:
num_solicitudes = 100000
motivos_rechazo_posibles = [
    'Falta de personal en esas fechas',
    'Superposición con otras vacaciones',
    'Solicitud fuera de plazo',
    'Motivo no justificado',
    'Antigüedad insuficiente para el periodo solicitado',
    'Rol crítico en el departamento durante esas fechas',
    'Exceso de solicitudes aprobadas en el departamento',
    'Prioridad baja en comparación con otras solicitudes',
    'Empleado con historial de ausencias frecuentes'
]
num_motivos = len(motivos_rechazo_posibles)
target_samples_per_motivo = 15000  # Define un número objetivo de muestras por motivo

vacaciones_data = []
rechazos_por_motivo = Counter({motivo: 0 for motivo in motivos_rechazo_posibles})

def calcular_prioridad(empleado, fecha_solicitud_date):
    score = 0
    score += empleado['antiguedad_anos'] * 0.5
    if empleado['personas_a_cargo'] > 0:
        score += 5
    dias_acumulados = min(empleado['antiguedad_anos'] * 8, 80)
    dias_disfrutados = random.randint(0, min(int(empleado['antiguedad_anos'] * 4), 10))
    if dias_disfrutados == 0 and empleado['antiguedad_anos'] >= 1:
        score += 10
    score += (dias_acumulados - dias_disfrutados) * 0.3
    if empleado['es_rol_critico']:
        score -= 10
    if empleado['tiene_ausencias_frecuentes']:
        score -= 5
    fecha_solicitud_datetime = datetime.combine(fecha_solicitud_date, datetime.min.time())
    dias_desde_solicitud = min((datetime.now() - fecha_solicitud_datetime).days * 0.05, 10)
    score += dias_desde_solicitud
    return score

with open('../data/vacaciones.csv', 'w', newline='', encoding='utf-8') as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(['id_empleado', 'vacaciones_inicio', 'vacaciones_fin', 'fecha_solicitud', 'dias_solicitados', 'estado_solicitud', 'motivo_rechazo', 'prioridad'])

    for _ in range(num_solicitudes):
        empleado = random.choice(empleados_data)
        fecha_solicitud_date = fake.date_between(start_date='-1y', end_date='-1w')
        fecha_inicio_date = fake.date_between(start_date=fecha_solicitud_date + timedelta(days=7), end_date='+6mo')
        delta = timedelta(days=random.randint(1, 21))
        fecha_fin_date = fecha_inicio_date + delta
        dias_solicitados = delta.days
        prioridad = calcular_prioridad(empleado, fecha_solicitud_date)

        estado_solicitud = 'aprobada'
        motivo_rechazo = None

        # Lógica para generar rechazos de forma más equilibrada
        if sum(rechazos_por_motivo.values()) < target_samples_per_motivo * num_motivos:
            if random.random() < 0.3:  # Aumentar la probabilidad general de rechazo
                motivo_seleccionado = random.choice([
                    m for m in motivos_rechazo_posibles if rechazos_por_motivo[m] < target_samples_per_motivo
                ])
                if motivo_seleccionado:
                    estado_solicitud = 'rechazada'
                    motivo_rechazo = motivo_seleccionado
                    rechazos_por_motivo[motivo_rechazo] += 1
        else:
            # Si ya se han generado suficientes rechazos, priorizar aprobaciones
            if random.random() < 0.8:
                estado_solicitud = 'aprobada'
            else:
                estado_solicitud = 'rechazada'
                motivo_rechazo = random.choice(motivos_rechazo_posibles) # Seguir generando rechazos, aunque menos probables

        # Lógica adicional para forzar algunos motivos basados en características
        if empleado['es_rol_critico'] and random.random() < 0.4 and rechazos_por_motivo['Rol crítico en el departamento durante esas fechas'] < target_samples_per_motivo:
            estado_solicitud = 'rechazada'
            motivo_rechazo = 'Rol crítico en el departamento durante esas fechas'
            rechazos_por_motivo[motivo_rechazo] += 1
        elif dias_solicitados > 15 and empleado['antiguedad_anos'] < 3 and random.random() < 0.5 and rechazos_por_motivo['Antigüedad insuficiente para el periodo solicitado'] < target_samples_per_motivo:
            estado_solicitud = 'rechazada'
            motivo_rechazo = 'Antigüedad insuficiente para el periodo solicitado'
            rechazos_por_motivo[motivo_rechazo] += 1
        elif prioridad < 30 and random.random() < 0.3 and rechazos_por_motivo['Prioridad baja en comparación con otras solicitudes'] < target_samples_per_motivo:
            estado_solicitud = 'rechazada'
            motivo_rechazo = 'Prioridad baja en comparación con otras solicitudes'
            rechazos_por_motivo[motivo_rechazo] += 1
        elif empleado['tiene_ausencias_frecuentes'] and random.random() < 0.2 and rechazos_por_motivo['Empleado con historial de ausencias frecuentes'] < target_samples_per_motivo:
            estado_solicitud = 'rechazada'
            motivo_rechazo = 'Empleado con historial de ausencias frecuentes'
            rechazos_por_motivo[motivo_rechazo] += 1
        elif (fecha_inicio_date - fecha_solicitud_date).days < 7 and random.random() < 0.2 and rechazos_por_motivo['Solicitud fuera de plazo'] < target_samples_per_motivo:
            estado_solicitud = 'rechazada'
            motivo_rechazo = 'Solicitud fuera de plazo'
            rechazos_por_motivo[motivo_rechazo] += 1
        elif random.random() < 0.05 and sum(rechazos_por_motivo.values()) < target_samples_per_motivo * num_motivos:
            otro_motivo = random.choice([
                m for m in ['Falta de personal en esas fechas', 'Superposición con otras vacaciones', 'Motivo no justificado', 'Exceso de solicitudes aprobadas en el departamento']
                if rechazos_por_motivo[m] < target_samples_per_motivo
            ])
            if otro_motivo:
                estado_solicitud = 'rechazada'
                motivo_rechazo = otro_motivo
                rechazos_por_motivo[motivo_rechazo] += 1

        vacaciones_data.append([empleado['id_empleado'], fecha_inicio_date.strftime('%Y-%m-%d'), fecha_fin_date.strftime('%Y-%m-%d'), fecha_solicitud_date.strftime('%Y-%m-%d'), dias_solicitados, estado_solicitud, motivo_rechazo, prioridad])

    # Escribir todos los datos al CSV
    writer.writerows(vacaciones_data)

print("El archivo 'vacaciones.csv' ha sido creado exitosamente con un intento de equilibrio de motivos de rechazo.")
print("Conteo de rechazos por motivo:", rechazos_por_motivo)



El archivo 'vacaciones.csv' ha sido creado exitosamente con un intento de equilibrio de motivos de rechazo.
Conteo de rechazos por motivo: Counter({'Rol crítico en el departamento durante esas fechas': 15000, 'Prioridad baja en comparación con otras solicitudes': 15000, 'Antigüedad insuficiente para el periodo solicitado': 6585, 'Empleado con historial de ausencias frecuentes': 5128, 'Exceso de solicitudes aprobadas en el departamento': 4407, 'Superposición con otras vacaciones': 4362, 'Falta de personal en esas fechas': 4317, 'Motivo no justificado': 4269, 'Solicitud fuera de plazo': 3418})
